In [194]:
import pyaudio

CHUNK = 4096
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 10

# audio_buffer=b''
audio_buffer=[]

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    # audio_buffer += data
    audio_buffer.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


In [195]:
audio_data_bytes = b''.join(audio_buffer)

In [196]:
len(audio_buffer)
# \x02\x00

107

In [118]:
import numpy as np

# Assuming your byte string is stored in audio_buffer
np.frombuffer(audio_data_bytes, dtype=np.int16)

array([ 0,  0, -1, ...,  4,  3,  3], dtype=int16)

In [119]:
len(np.frombuffer(audio_data_bytes, dtype=np.int16))

212992

# Check if speech is there in each chunk

In [198]:
from silero_vad import SileroVAD
import soundfile as sf
import numpy as np
import tempfile
import os

# Hardcode RATE
RATE=44100

# Initialize SileroVAD
onnx_model = '../using-fastapi/silero-vad-basics/silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

speech_segments = []

# Total number of samples processed so far
total_samples = 0

for i in range(len(audio_buffer)):

    audio_array = np.frombuffer(audio_buffer[i], dtype=np.int16)

    # Create a temporary WAV file to store the audio data
    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmpfile:
        sf.write(tmpfile.name,audio_array, RATE)
        tmpfile_path = tmpfile.name

    # Get speech timestamps for the temporary WAV file
    speech_timestamps = vad.get_speech_timestamps(tmpfile_path)

    for speech in speech_timestamps:
        # Adjust the timestamps relative to the entire audio stream
        start = speech['start'] + total_samples
        end = speech['end'] + total_samples
        print('chunk', i, 'contains speech', {'start': start, 'end': end})
        speech_segments.append(audio_array[speech['start']:speech['end']])
    
    total_samples += len(audio_array)

    # Clean up the temporary file
    os.unlink(tmpfile_path)



In [201]:
speech_segments

[]

In [199]:
sf.write('test.wav',np.concatenate(speech_segments),44100)

ValueError: need at least one array to concatenate

# Apply VAD to entire audio array

In [197]:
# Initialize SileroVAD
onnx_model = '../using-fastapi/silero-vad-basics/silero_vad.onnx'
vad = SileroVAD(onnx_model=onnx_model)

speech_segments = []

with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmpfile:
    audio_array = np.frombuffer(audio_data_bytes, dtype=np.int16)
    sf.write(tmpfile.name,audio_array, RATE)
    tmpfile_path = tmpfile.name

# Get speech timestamps for the temporary WAV file
speech_timestamps = vad.get_speech_timestamps(tmpfile_path)
for speech in speech_timestamps:
    print('contains speech', speech)
    speech_segments.append(audio_array[speech['start']:speech['end']])

sf.write('test.wav',np.concatenate(speech_segments),44100)

contains speech {'start': 55125, 'end': 74705}
contains speech {'start': 91816, 'end': 115630}
contains speech {'start': 138385, 'end': 159377}
contains speech {'start': 183544, 'end': 205947}
contains speech {'start': 232936, 'end': 255339}
contains speech {'start': 276683, 'end': 301908}
contains speech {'start': 330309, 'end': 355534}
contains speech {'start': 417803, 'end': 438272}
